<a href="https://colab.research.google.com/github/objectc/CNN-with-TensorFlow2.0-and-Keras/blob/master/tansfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [2]:
# For the bug of Keras: Incompatible shapes when using accuracy metric, sparse_categorical_crossentropy, and batch size > 1
# Please visit https://github.com/keras-team/keras/issues/11749 for more information
!pip3 install git+https://github.com/keras-team/keras.git -U 

  Cloning https://github.com/keras-team/keras.git to /tmp/pip-req-build-vx8_w8z9
  Running command git clone -q https://github.com/keras-team/keras.git /tmp/pip-req-build-vx8_w8z9
  Stored in directory: /tmp/pip-ephem-wheel-cache-fwke92b4/wheels/da/a4/7e/6b7bd9af18cc2e23b8dd5ed6de07a7e13bd80a17214eb88932
Successfully built Keras
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [0]:
# Import all the necessary files!
import tensorflow as tf
from tensorflow import keras
import sklearn
import matplotlib.pyplot as plt

In [0]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [5]:
# Download the inception v3 weights
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5


--2019-05-16 00:00:47--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   132MB/s    in 0.6s    

2019-05-16 00:00:48 (132 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
local_weight_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (96, 96, 3), include_top = False, weights = None)


In [0]:
pre_trained_model.load_weights(local_weight_file)
for layer in pre_trained_model.layers:
  layer.trainable = False

In [8]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 47, 47, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 47, 47, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 47, 47, 32)   0           batch_normalization_v1[0][0]     
_______________________________________________________________________________________

In [0]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [0]:
X_train_full = X_train_full/255.0
X_test = X_test/255.0

In [27]:
X_test.shape

(10000, 32, 32, 3)

In [28]:
X_train_full.shape

(50000, 32, 32, 3)

In [0]:
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [12]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
immediateModel = keras.Model(pre_trained_model.inputs, last_layer.output)
# last_layer_fuc = keras.backend.function(pre_trained_model.inputs, last_layer.output)

last layer output shape:  (None, 4, 4, 768)


In [0]:
keras.backend.clear_session()
train_input = keras.layers.Input((32, 32, 3))
x = keras.layers.Lambda(lambda image: tf.image.resize(image, (96, 96)))(train_input)
x = immediateModel(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(4096, activation='relu')(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(len(classes), activation=keras.activations.softmax)(x)

In [0]:
model = keras.Model(train_input, x)

In [0]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [37]:
model.compile(optimizer=keras.optimizers.Adam(lr=lr_schedule(0)), 
              loss=keras.losses.sparse_categorical_crossentropy, 
              metrics=[keras.metrics.sparse_categorical_accuracy])

Learning rate:  0.001


In [0]:
from keras_preprocessing.image import ImageDataGenerator

In [0]:
traingen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format='channels_last',
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)
traingen.fit(X_train_full)
X_train_gen = traingen.flow(X_train_full, y_train_full)

In [0]:
def plot_history(history):
  # Plot training & validation accuracy values
  plt.plot(history.history['sparse_categorical_accuracy'])
  plt.plot(history.history['val_sparse_categorical_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validate'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Validate'], loc='upper left')
  plt.show()

In [0]:
checkpoints_cb = keras.callbacks.ModelCheckpoint('./checkpoints/weights{epoch:08d}.h5', 
                                     save_weights_only=True, period=10)
stop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
lr_reducer = keras.callbacks.ReduceLROnPlateau(factor= 0.1**0.5,
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoints_cb, lr_reducer, lr_scheduler, stop_cb]

In [42]:
history = model.fit(X_train_gen, epochs=100, batch_size=128,validation_data=(X_test, y_test), callbacks=callbacks)

Learning rate:  0.001
Epoch 1/100
1563/1563 [==============================] - 71s 46ms/step - loss: 0.9735 - sparse_categorical_accuracy: 0.6720 - val_loss: 0.6874 - val_sparse_categorical_accuracy: 0.7664
Learning rate:  0.001
Epoch 2/100
1563/1563 [==============================] - 66s 42ms/step - loss: 0.7745 - sparse_categorical_accuracy: 0.7370 - val_loss: 0.6399 - val_sparse_categorical_accuracy: 0.7831
Learning rate:  0.001
Epoch 3/100
1563/1563 [==============================] - 65s 42ms/step - loss: 0.7110 - sparse_categorical_accuracy: 0.7582 - val_loss: 0.6297 - val_sparse_categorical_accuracy: 0.7848
Learning rate:  0.001
Epoch 4/100
1563/1563 [==============================] - 67s 43ms/step - loss: 0.6698 - sparse_categorical_accuracy: 0.7709 - val_loss: 0.6342 - val_sparse_categorical_accuracy: 0.7961
Learning rate:  0.001
Epoch 5/100
1563/1563 [==============================] - 69s 44ms/step - loss: 0.6514 - sparse_categorical_accuracy: 0.7802 - val_loss: 0.6155 - val_s

In [0]:
plot_history(history)

In [0]:
model.save('transfer_model.h5')

In [0]:
model.save_weights('transfer_weights.h5')